#  <center>Project 1 - Baseline implementation</center>

The objective of this project is to test different architectures to compare two digits visible in a
two-channel image. It aims at showing in particular the impact of weight sharing, and of the use of an
auxiliary loss to help the training of the main objective.
It should be implemented with PyTorch only code, in particular without using other external libraries
such as scikit-learn or numpy.


In [6]:
import torch
from torch import Tensor
from torch import nn
from torch.nn import functional as F
import dlc_practical_prologue as prologue

# Loading the data

We generate the data sets to using with the function generate_pair_sets(N) defined in the file
dlc_practical_prologue.py. This function returns six tensors:


| Name          | Tensor dimension | Type     | Content                                    |
|---------------|------------------|----------|--------------------------------------------|
| train_input   | N × 2 × 14 × 14  | float32  | Images                                     |
| train_target  | N                | int64    | Class to predict ∈ {0, 1}                  |
| train_classes | N × 2            | int64    | Classes of the two digits ∈ {0, . . . , 9} |
| test_input    | N × 2 × 14 × 14  | float32  | Images                                     |
| test_target   | N                | int64    | Class to predict ∈ {0, 1}                  |
| test_classes  | N × 2            | int64    | Classes of the two digits ∈ {0, . . . , 9} |

In [130]:
train_input, train_target, train_classes, test_input, test_target, test_classes = prologue.generate_pair_sets(1000)

In [ ]:
mu,std = train_input.mean(), train_input.std()
train_input.sub_(mu).div_(std)

mu,std = test_input.mean(), test_input.std()
test_input.sub_(mu).div_(std)

# Defining the Model

In [202]:
class Net(nn.Module):
    def __init__(self, nb_hidden):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(2, 32, kernel_size=3)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.fc1 = nn.Linear(256, nb_hidden)
        self.fc2 = nn.Linear(nb_hidden, 2)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size=2, stride=2))
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size=2, stride=2))
        x = F.relu(self.fc1(x.view(-1, 256)))
        x = self.fc2(x)
        return x


# Training the model

CNN with 2 convolutional layers and 2 fully connected layers 

In [209]:
batch_size = 100
n_epochs = 250
def train_model(model,train_input,train_target):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(),lr = 0.1)
    for e in range(0,n_epochs):
        for input, targets in zip(train_input.split(batch_size),train_target.split(batch_size)):
            output = model(input)
            loss = criterion(output,targets)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        if(e%50 ==0):
            print('epoch : ',e,' loss : ',loss.item())

In [211]:
model = Net(100)
train_model(model,train_input,train_target)

epoch :  0  loss :  0.6855924129486084
epoch :  50  loss :  0.0034624480176717043
epoch :  100  loss :  0.0008349684067070484
epoch :  150  loss :  0.0004290234064683318
epoch :  200  loss :  0.000278322899248451


# Testing

In [210]:
def compute_nb_errors(model,data_input,data_target):
    nb_errors = 0
    for input,targets in zip(data_input.split(batch_size),data_target.split(batch_size)):
        output = model(input)
        _,predicted_classes = torch.max(output,1)
        for i in range(0,output.size(0)):
            if(predicted_classes[i]!=targets[i]):
                nb_errors = nb_errors+1
                
    return nb_errors

In [216]:
print('train_error {:.02f}% test_error {:.02f}%'.format(
    compute_nb_errors(model, train_input, train_target) / train_input.size(0) * 100,
    compute_nb_errors(model, test_input, test_target) / test_input.size(0) * 100))


train_error 0.00% test_error 18.70%
